In [ ]:
# Imports and configuration
import os
import subprocess

# User-defined parameters
PROT = "6KWC"
TRI_RESIDUE_IDX = 18

# Input/output paths
BASE_DATA_DIR = ""  # path to AlphaFold data
FASTA_DIR = "" # directory containing fasta file to evaluate
ATTN_MAP_DIR = "" # directory for saving text files with top-k attention scores
ALIGNMENT_DIR = "" # directory containing pre-computed alignment files (and MSAs)
OUTPUT_DIR = "" # directory to save outputs


In [2]:
# Run OpenFold inference and save top attention scores to text files 
inference_cmd = f"""
python3 run_pretrained_openfold.py \
    {FASTA_DIR} \
    {BASE_DATA_DIR}/pdb_mmcif/mmcif_files \
    --use_precomputed_alignments {ALIGNMENT_DIR} \
    --output_dir {OUTPUT_DIR} \
    --config_preset model_1_ptm \
    --uniref90_database_path {BASE_DATA_DIR}/uniref90/uniref90.fasta \
    --mgnify_database_path {BASE_DATA_DIR}/mgnify/mgy_clusters_2022_05.fa \
    --pdb70_database_path {BASE_DATA_DIR}/pdb70/pdb70 \
    --uniclust30_database_path {BASE_DATA_DIR}/uniclust30/uniclust30_2018_08 \
    --bfd_database_path {BASE_DATA_DIR}/bfd/bfd_metaclust_clu_complete_id30_c90_final_seq.sorted_opt \
    --save_outputs \
    --model_device "cuda:0" \
    --attn_map_dir {ATTN_MAP_DIR} \
    --num_recycles_save 1 \
    --triangle_residue_idx {TRI_RESIDUE_IDX} \
    --demo_attn
"""

subprocess.run(inference_cmd, shell=True, check=True)


CUDA available:  True
GPU name:  NVIDIA A100-SXM4-40GB


/u/thayes/anaconda3/envs/openfold_env2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


args: 
Namespace(fasta_dir='/u/thayes/vizfold/examples/monomer/fasta_dir_6KWC', template_mmcif_dir='/ime/hdd/rhaas/SUP-5301/database/pdb_mmcif/mmcif_files', use_precomputed_alignments='/projects/bekh/thayes/output_dirs/my_outputs_align_6KWC/alignments', use_custom_template=False, use_single_seq_mode=False, output_dir='/projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18', model_device='cuda:0', config_preset='model_1_ptm', jax_param_path=None, openfold_checkpoint_path=None, save_outputs=True, cpus=4, preset='full_dbs', output_postfix=None, data_random_seed=None, skip_relaxation=False, multimer_ri_gap=200, trace_model=False, subtract_plddt=False, long_sequence_inference=False, cif_output=False, experiment_config_json='', use_deepspeed_evoformer_attention=False, attn_map_dir='/projects/bekh/thayes/demo/6KWC_demo/', num_recycles_save=1, demo_attn=True, triangle_residue_idx=18, uniref90_database_path='/ime/hdd/rhaas/SUP-5301/database/uniref90/uniref90.fasta', mgnify_database

INFO:/u/thayes/openfold/openfold/utils/script_utils.py:Successfully loaded JAX parameters at openfold/resources/params/params_model_1_ptm.npz...
INFO:/u/thayes/openfold/run_pretrained_openfold.py:Using precomputed alignments for 6KWC_1 at /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC/alignments...
INFO:/u/thayes/openfold/openfold/utils/script_utils.py:Running inference for 6KWC_1...


RECYCLING STEP  0
	evo_block_0_recycle_0
[Done] Saved top 500 entries for msa_row_attn to /projects/bekh/thayes/demo/6KWC_demo/msa_row_attn_layer0.txt
[Done] Saved top 500 entries for triangle_start_attn to /projects/bekh/thayes/demo/6KWC_demo/triangle_start_attn_layer0_residue_idx_18.txt
	evo_block_1_recycle_0
[Done] Saved top 500 entries for msa_row_attn to /projects/bekh/thayes/demo/6KWC_demo/msa_row_attn_layer1.txt
[Done] Saved top 500 entries for triangle_start_attn to /projects/bekh/thayes/demo/6KWC_demo/triangle_start_attn_layer1_residue_idx_18.txt
	evo_block_2_recycle_0
[Done] Saved top 500 entries for msa_row_attn to /projects/bekh/thayes/demo/6KWC_demo/msa_row_attn_layer2.txt
[Done] Saved top 500 entries for triangle_start_attn to /projects/bekh/thayes/demo/6KWC_demo/triangle_start_attn_layer2_residue_idx_18.txt
	evo_block_3_recycle_0
[Done] Saved top 500 entries for msa_row_attn to /projects/bekh/thayes/demo/6KWC_demo/msa_row_attn_layer3.txt
[Done] Saved top 500 entries for 

INFO:/u/thayes/openfold/openfold/utils/script_utils.py:Inference time: 29.974448291002773
INFO:/u/thayes/openfold/run_pretrained_openfold.py:Output written to /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18/predictions/6KWC_1_model_1_ptm_unrelaxed.pdb...
INFO:/u/thayes/openfold/run_pretrained_openfold.py:Running relaxation on /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18/predictions/6KWC_1_model_1_ptm_unrelaxed.pdb...
INFO:/u/thayes/openfold/openfold/utils/script_utils.py:Relaxation time: 6.697049687965773
INFO:/u/thayes/openfold/openfold/utils/script_utils.py:Relaxed output written to /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18/predictions/6KWC_1_model_1_ptm_relaxed.pdb...
INFO:/u/thayes/openfold/run_pretrained_openfold.py:Model output written to /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18/predictions/6KWC_1_model_1_ptm_output_dict.pkl...


CompletedProcess(args='\npython3 run_pretrained_openfold.py     /u/thayes/vizfold/examples/monomer/fasta_dir_6KWC     /ime/hdd/rhaas/SUP-5301/database/pdb_mmcif/mmcif_files     --use_precomputed_alignments /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC/alignments     --output_dir /projects/bekh/thayes/output_dirs/my_outputs_align_6KWC_demo_tri_18     --config_preset model_1_ptm     --uniref90_database_path /ime/hdd/rhaas/SUP-5301/database/uniref90/uniref90.fasta     --mgnify_database_path /ime/hdd/rhaas/SUP-5301/database/mgnify/mgy_clusters_2022_05.fa     --pdb70_database_path /ime/hdd/rhaas/SUP-5301/database/pdb70/pdb70     --uniclust30_database_path /ime/hdd/rhaas/SUP-5301/database/uniclust30/uniclust30_2018_08     --bfd_database_path /ime/hdd/rhaas/SUP-5301/database/bfd/bfd_metaclust_clu_complete_id30_c90_final_seq.sorted_opt     --save_outputs     --model_device "cuda:0"     --attn_map_dir /projects/bekh/thayes/demo/6KWC_demo/     --num_recycles_save 1     --triangle_resid

In [ ]:
# Generate attention arc diagrams (MSA + Triangle Start)
from visualize_attention_arc_diagram_demo_utils import generate_arc_diagrams, parse_fasta_sequence
arc_output_dir_msa = "" # directory for saving msa attention arc diagrams
arc_output_dir_tri = "" # directory for saving triangle attention arc diagrams
FASTA_PATH = "" # fasta file for labeling plots
LAYER_IDX = 47 # selected layer for attention evaluation
TOP_K = 50 # top-k lines to plot (limit to 500)

residue_seq = parse_fasta_sequence(FASTA_PATH)

generate_arc_diagrams(
    attention_dir=ATTN_MAP_DIR,
    residue_sequence=residue_seq,
    output_dir=arc_output_dir_msa,
    attention_type="msa_row",
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)

generate_arc_diagrams(
    attention_dir=ATTN_MAP_DIR,
    residue_sequence=residue_seq,
    output_dir=arc_output_dir_tri,
    attention_type="triangle_start",
    residue_indices=[TRI_RESIDUE_IDX],
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)


[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_0_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_1_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_2_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_3_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_4_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_5_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_6_arc.png
[Saved] /u/thayes/vizfold/demo_arc_msa_row_NEW/msa_row_head_7_arc.png
[Saved] /u/thayes/vizfold/demo_arc_tri_start_NEW/tri_start_res_18_head_0_arc.png
[Saved] /u/thayes/vizfold/demo_arc_tri_start_NEW/tri_start_res_18_head_1_arc.png
[Saved] /u/thayes/vizfold/demo_arc_tri_start_NEW/tri_start_res_18_head_2_arc.png
[Saved] /u/thayes/vizfold/demo_arc_tri_start_NEW/tri_start_res_18_head_3_arc.png


In [ ]:
# Generate PyMOL 3D attention visualizations
from visualize_attention_3d_demo_utils import plot_pymol_attention_heads
output_dir_msa = "" # directory for saving msa attention 3D visuals
output_dir_tri = "" # directory for saving triangle attention 3D visuals
PDB_FILE = "" # PDB file for plotting in 3D
LAYER_IDX = 47 # selected layer for attention evaluation
TOP_K = 50 # top-k lines to plot (limit to 500)

plot_pymol_attention_heads(
    pdb_file=PDB_FILE,
    attention_dir=ATTN_MAP_DIR,
    output_dir=output_dir_msa,
    attention_type="msa_row",
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)

plot_pymol_attention_heads(
    pdb_file=PDB_FILE,
    attention_dir=ATTN_MAP_DIR,
    output_dir=output_dir_tri,
    attention_type="triangle_start",
    residue_indices=[TRI_RESIDUE_IDX],
    top_k=TOP_K,
    layer_idx=LAYER_IDX
)


: 